In [1]:
import gdown
import zipfile
import os

# Google Drive file ID and URL
file_id = "1EEPJeNoL_r9nVTCyCEKNiBx371dG_SJx"
url = f"https://drive.google.com/uc?id={file_id}"

# Destination file path
output_path = "data_split.zip"

# Download the file from Google Drive
gdown.download(url, output_path, quiet=False)

# Extract the ZIP file
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall("./")  # Extract into the current directory

# List all extracted directories
dirs = [d for d in os.listdir() if os.path.isdir(d)]

# Filter out only numeric directories and sort them numerically
numeric_dirs = [d for d in dirs if d.isdigit()]
sorted_dirs = sorted(numeric_dirs, key=lambda x: int(x))

# Optionally, rename directories to ensure they are in numerical order
for i, d in enumerate(sorted_dirs):
    os.rename(d, str(i))  # Rename to '0', '1', '2', etc.

print("Folders have been renamed and sorted numerically.")

Downloading...
From (original): https://drive.google.com/uc?id=1EEPJeNoL_r9nVTCyCEKNiBx371dG_SJx
From (redirected): https://drive.google.com/uc?id=1EEPJeNoL_r9nVTCyCEKNiBx371dG_SJx&confirm=t&uuid=ef6e0f2c-94c3-4566-ad30-7a8a9cf13390
To: /content/data_split.zip
100%|██████████| 47.3M/47.3M [00:00<00:00, 95.2MB/s]


Folders have been renamed and sorted numerically.


In [1]:
#pytoch data loader
import torch.optim as optim
import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn

# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Paths to splits
train_dir = "/content/data_split/train"
val_dir = "/content/data_split/validation"
test_dir = "/content/data_split/test"

# Image dimensions and batch size
img_size = (224, 224)
batch_size = 32

train_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f'Train dataset size: {len(train_dataset)}')
print(f'Validation dataset size: {len(val_dataset)}')
print(f'Test dataset size: {len(test_dataset)}')

Train dataset size: 2730
Validation dataset size: 572
Test dataset size: 598


In [3]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 26)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.batch_norm = nn.BatchNorm1d(512)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = self.pool(nn.ReLU()(self.conv3(x)))
        x = x.view(-1, 256 * 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.batch_norm(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [4]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device=device):
    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_accuracy = correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Accuracy: {train_accuracy*100}%")

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)

        val_accuracy = val_correct / val_total
        print(f"Validation Accuracy: {val_accuracy*100}%")

    return model

In [5]:
best_batch_size = 32
best_learning_rate = 0.001
best_weight_decay = 0.0001

model = CustomCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=best_learning_rate, weight_decay=best_weight_decay)

train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False)

epochs = 10
criterion = nn.CrossEntropyLoss()

final_trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, epochs)

Epoch 1/10, Loss: 0.4932769084955717, Accuracy: 90.76923076923077%
Validation Accuracy: 100.0%
Epoch 2/10, Loss: 0.013760132834228665, Accuracy: 100.0%
Validation Accuracy: 100.0%
Epoch 3/10, Loss: 0.006394736818037927, Accuracy: 100.0%
Validation Accuracy: 100.0%
Epoch 4/10, Loss: 0.003082443941059674, Accuracy: 100.0%
Validation Accuracy: 100.0%
Epoch 5/10, Loss: 0.006184950565681035, Accuracy: 100.0%
Validation Accuracy: 100.0%
Epoch 6/10, Loss: 0.015976187800122207, Accuracy: 99.8901098901099%
Validation Accuracy: 100.0%
Epoch 7/10, Loss: 0.00611541896441215, Accuracy: 99.92673992673993%
Validation Accuracy: 100.0%
Epoch 8/10, Loss: 0.0022680495099467765, Accuracy: 100.0%
Validation Accuracy: 100.0%
Epoch 9/10, Loss: 0.0012058676589350758, Accuracy: 100.0%
Validation Accuracy: 100.0%
Epoch 10/10, Loss: 0.0006567969434992085, Accuracy: 100.0%
Validation Accuracy: 100.0%


In [7]:
# Save the trained model
torch.save(final_trained_model.state_dict(), "custom_cnn_model.pth")